In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime 

In [2]:
IN_file = "/Users/mquinton/Desktop/ETL-Project-Group5/Resources/IN_COVID.csv"
IN_df = pd.read_csv(IN_file)
IN_df.head()

,Date,New Cases,7-Day Moving Avg
0,Dec 1 2020,5396,5405
1,Nov 30 2020,5665,5437
2,Nov 29 2020,4304,5422
3,Nov 28 2020,4471,5689
4,Nov 27 2020,5643,6032


- I chose this dataset from the CDC by selecting Indiana and downloading the csv file that shows new cases compared to 7 day moving average by date. The reason we selected this dataset was to compare the data to our Google mobility dataset. We hope to compare cases and where people were going to see if there is a correlation or a causation to the rise in cases.
- Check date format

In [3]:
IN_df.dtypes

Date                object
New Cases            int64
7-Day Moving Avg     int64
dtype: object

- Needed to get data types to see what the date column was so I can convert the format

In [4]:
# IN_df = pd.to_datetime(IN_df('Date', format='%Y%m%d', errors='ignore'))

In [7]:
# renamed columns
IN_df.rename(columns={'New Cases': 'New_Cases',
                      '7-Day Moving Avg': 'Seven_Day_Moving_Avg'
                     }
                      )

,Date,New_Cases,Seven_Day_Moving_Avg
0,Dec 1 2020,5396,5405
1,Nov 30 2020,5665,5437
2,Nov 29 2020,4304,5422
3,Nov 28 2020,4471,5689
4,Nov 27 2020,5643,6032
...,...,...,...
310,Jan 26 2020,0,0
311,Jan 25 2020,0,0
312,Jan 24 2020,0,0
313,Jan 23 2020,0,0


In [8]:
# Set dataframe index to the date
IN_df.set_index('Date')

,New Cases,7-Day Moving Avg
Date,,
Dec 1 2020,5396,5405
Nov 30 2020,5665,5437
Nov 29 2020,4304,5422
Nov 28 2020,4471,5689
Nov 27 2020,5643,6032
...,...,...
Jan 26 2020,0,0
Jan 25 2020,0,0
Jan 24 2020,0,0


In [11]:
# Sort dataframe by date - display the last 30 days reported
IN_df.sort_index(ascending=False)
IN_df.set_index('Date').head(30)

,New Cases,7-Day Moving Avg
Date,,
Dec 1 2020,5396,5405
Nov 30 2020,5665,5437
Nov 29 2020,4304,5422
Nov 28 2020,4471,5689
Nov 27 2020,5643,6032
Nov 26 2020,6373,6198
Nov 25 2020,5983,6328
Nov 24 2020,5625,6333
Nov 23 2020,5556,6309
